In [1]:
from tqdm import tqdm
import numpy as np
from copy import deepcopy

## Part 1

In [2]:
# given two map lines, the first with ' ' in place of the pipes that belong to the cycle, and the second being the original
# one, returns the number of chars that are inside the cycle and not part of it
# if return_line, it also outputs a string with the inner points filled with '#'
def count_inside_points(line: str, return_line =  False) -> int:
    counter = 0
    is_inside = False
    new_line = ''
    # these are to deal with the cases FJ, l7 -> have a crossing, and F7, LJ -> do not have a crossing
    convex = False
    concave = False
    is_diagonal = False
    for char in line:
        if char == '|': # this surely makes us cross it
                is_inside = not is_inside
                counter += 1
            # these two variables count the parity of convex/concave bumps on the line
            # in my map, S is an 'L'. It can be computed automatically and passed to this function
        elif char == '-':
                counter +=1
        elif char in 'LJ':
                concave = not concave
                counter += 1
        elif char in 'F7':
                convex = not convex
                counter += 1
            # convex + concave -> straight diagonal line, counts as crossing
        if concave and convex:
                is_inside = not is_inside
                concave = False
                convex = False
        # if it does not belong to the cycle and we are inside, add 1        
        if char == '.':
            if is_inside:
                counter += 1
                new_line += '#'
            else:
                new_line += '.'
        else:
            new_line += char
                
    if return_line:        
        return counter, new_line
    else:
        return counter

In [3]:
instructions = []

# Part 1 is a copy by Day 10 
# from previous data check
n_cols = 1236 
n_rows = 1171

total_map = np.array([['.' for xdx in range(n_cols)] for ydx in range(n_rows)])

current_pos = [int(1171/2), int(1236/2)]
previous_dir = 'U'
current_dir = ''

# printing the border, and using FL7J to mark the corners, so that I can use the same function from Day 10
with open('Day18_input.txt') as f:
    for line in f:
        line = line.strip('\n').split(' ')
        line[2] = line[2][1:-1]
        n_steps = int(line[1])
        current_dir = line[0]
        match current_dir:
            case 'R':
                if previous_dir == 'U':
                    total_map[current_pos[0]][current_pos[1]] = 'F'
                else: # 'D'
                    total_map[current_pos[0]][current_pos[1]] = 'L'
                for xdx in range(current_pos[1]+1, current_pos[1] + n_steps):
                    total_map[current_pos[0]][xdx] = '-'
                current_pos = [current_pos[0], current_pos[1] + n_steps]
                
            case 'L':
                if previous_dir == 'U':
                    total_map[current_pos[0]][current_pos[1]] = '7'
                else: # 'D'
                    total_map[current_pos[0]][current_pos[1]] = 'J'
                for xdx in range(current_pos[1]-1, current_pos[1] - n_steps, -1):
                    total_map[current_pos[0]][xdx] = '-'
                current_pos = [current_pos[0],current_pos[1] - n_steps ]

            case 'U':
                if previous_dir == 'L':
                    total_map[current_pos[0]][current_pos[1]] = 'L'
                else: # 'R'
                    total_map[current_pos[0]][current_pos[1]] = 'J'                    
                for ydx in range(current_pos[0] -1, current_pos[0] -n_steps,-1):
                    total_map[ydx][current_pos[1]] = '|'                    
                current_pos = [current_pos[0] - n_steps, current_pos[1]]
                
            case 'D':
                if previous_dir == 'L':
                    total_map[current_pos[0]][current_pos[1]] = 'F'
                else: # 'R'
                    total_map[current_pos[0]][current_pos[1]] = '7'
                for ydx in range(current_pos[0]+1, current_pos[0] + n_steps):
                    total_map[ydx][current_pos[1]] = '|'
                current_pos = [current_pos[0] + n_steps, current_pos[1]]  
                 
        previous_dir = current_dir
            
              
counter = 0
# for visualization
#with open('Day18_map.txt', 'w') as f:
#    for line in total_map:
#        value, new_line = count_inside_points(line, return_line=True)
#        counter += value
##        f.write(''.join(line) +'\n')
#        f.write(new_line +'\n')
    
for line in total_map:
    value, new_line = count_inside_points(line, return_line=False)
    counter += value
print(counter)






 [['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ...
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']]
62


## Part 2

In [45]:
direction_dict ={0: 'R',
                 1: 'D',
                 2: 'L',
                 3: 'U'
                 }

# have to cast everything manually into np.int64 so I do not get overflow problems
current_pos = np.array(np.int64([0, 0]))
edges= [deepcopy(current_pos)]


inner_area =  np.int64(0)
perimeter = np.int64(0)
with open('Day18_input.txt') as f:
    for line in f:
        line = line.strip('\n').split(' ')
        # with these two lines uncommented, it solves Part 1 instead
        #n_steps = int(line[1])
        #current_dir = line[0]
        n_steps =  int(line[2][2:-2], 16)
        current_dir = direction_dict[int(line[2][-2])]
        # count perimeter for Pick's theorem
        perimeter += n_steps
        match current_dir:
            case 'R':
                current_pos += [0,n_steps]
            case 'L':
                current_pos += [0, -n_steps]
            case 'U':
                current_pos += [-n_steps, 0]
            case 'D':
                current_pos += [n_steps, 0]
        # count shoelaces terms for shoelace formula for internal points
        shoelace = np.int64(-(edges[-1][1]*current_pos[0] - edges[-1][0]*current_pos[1] ))
        inner_area +=shoelace
        # I don't even need to keep track of all edges, but it was needed for debugging
        edges.append(deepcopy(current_pos))
        
        
print('final area is ',int(abs(inner_area)/2+perimeter/2+1))


 L 175246
[1000 1000]
-175246000 -175246000

 D 84855
[   1000 -174246]
14785644330 14610398330

 L 197216
[  85855 -174246]
-16931979680 -2321581350

 U 84855
[  85855 -371462]
-31520408010 -33841989360

 L 351096
[   1000 -371462]
-351096000 -34193085360

 U 341282
[   1000 -722558]
-246596039356 -280789124716

 L 245512
[-340282 -722558]
83543314384 -197245810332

 U 507459
[-340282 -968070]
-491255834130 -688501644462

 R 311558
[-847741 -968070]
-264120490478 -952622134940

 U 239259
[-847741 -656512]
-157076404608 -1109698539548

 R 389494
[-1087000  -656512]
-423379978000 -1533078517548

 U 426796
[-1087000  -267018]
-113962214328 -1647040731876

 R 192726
[-1513796  -267018]
-291747847896 -1938788579772

 D 63173
[-1513796   -74292]
4693248516 -1934095331256

 L 172292
[-1450623   -74292]
249930737916 -1684164593340

 D 287982
[-1450623  -246584]
71011753488 -1613152839852

 R 172292
[-1162641  -246584]
-200313743172 -1813466583024

 D 314900
[-1162641   -74292]
23394550800 -1

In [ ]:
test = 952408144115 
mine = 952408144115

In [24]:
print(edges)

print(100000*561937 -100000*100000)

test = [[ 100000, 100000], [ 100000, 561937]]
shoelace = -(test[-1][1]*test[-2][0] - test[-1][0]*test[-2][1] )
print(-shoelace)
print(1050940256)

[array([100000, 100000]), array([100000, 561937]), array([156407, 561937]), array([156407, 918608]), array([1019647,  918608]), array([1019647, 1286328]), array([1286328, 1286328]), array([1286328,  709066]), array([456353, 709066]), array([456353, 597056]), array([1286328,  597056]), array([1286328,  105411]), array([600254, 105411]), array([600254, 100000]), array([100000, 100000])]
46193700000
46193700000
1050940256


In [41]:
print(edges)
inner = np.int64(0)
shoes = []
for edge1, edge2 in zip(edges[:-1], edges[1:]):
    shoe = edge1[0]*edge2[1] - edge1[1]*edge2[0]
    print('partial sum ', inner, ' + shoelace term ', shoe, ' = ', inner+shoe, type(inner+shoe))
    inner += shoe
    shoes.append(shoe)
    
print(sum(shoes))

[array([1000, 1000]), array([  1000, 462937]), array([ 57407, 462937]), array([ 57407, 819608]), array([920647, 819608]), array([ 920647, 1187328]), array([1187328, 1187328]), array([1187328,  610066]), array([357353, 610066]), array([357353, 498056]), array([1187328,  498056]), array([1187328,    6411]), array([501254,   6411]), array([501254,   1000]), array([1000, 1000])]
partial sum  0  + shoelace term  461937000  =  461937000 <class 'numpy.int64'>
partial sum  461937000  + shoelace term  -343083583  =  118853417 <class 'numpy.int64'>
partial sum  118853417  + shoelace term  -999424383  =  -880570966 <class 'numpy.int64'>
partial sum  -880570966  + shoelace term  1151193920  =  270622954 <class 'numpy.int64'>
partial sum  270622954  + shoelace term  -762101544  =  -491478590 <class 'numpy.int64'>
partial sum  -491478590  + shoelace term  1189761536  =  698282946 <class 'numpy.int64'>
partial sum  698282946  + shoelace term  1795431424  =  2493714370 <class 'numpy.int64'>
partial su

C:\Users\morellil\AppData\Local\Temp\ipykernel_19148\2837080429.py:5: RuntimeWarning: overflow encountered in scalar multiply
  shoe = edge1[0]*edge2[1] - edge1[1]*edge2[0]
C:\Users\morellil\AppData\Local\Temp\ipykernel_19148\2837080429.py:5: RuntimeWarning: overflow encountered in scalar subtract
  shoe = edge1[0]*edge2[1] - edge1[1]*edge2[0]
C:\Users\morellil\AppData\Local\Temp\ipykernel_19148\2837080429.py:10: RuntimeWarning: overflow encountered in scalar add
  print(sum(shoes))


In [35]:
1582681902 + 500254000

2082935902